# Creating your own EmissionModel

To create your own `EmissionModel` all you need to do is define each individual emission type and relate them. This operation will define a tree-like structure, where each model links to at least one other model. 

In the sections below we will detail how to define each different type of emission operation, but there are a few arguments needed by an ``EmissionModel`` regardless of the operation:

- A ***label*** which identifies the model. This is given by passing a string to the ``label`` argument. These must be unique within a single emission model "tree".
- An ***emitter*** to act on. This is given by passing a string to the ``emitter`` argument. Currently the possible emitters are: ``"stellar"``, ``"blackhole"``, and ``"galaxy"``. 

Before demonstrating anything we first need to import the classes we'll need and set up a ``Grid`` (described [here](../emission_grids/grids.rst)).

In [ ]:
from unyt import cm, dimensionless, kelvin, km, s

from synthesizer.emission_models import EmissionModel, Greybody
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.grid import Grid

# Get the grid which we'll need for extraction
grid_name = "test_grid"
grid = Grid(grid_name)


## Emitters

The emitter on a model defines: 

- Where any model independant attributes should be extracted from (e.g. ages and metallicites needed for stellar extractions).
- Where the resulting spectra/lines will be stored once generated. 
- Where any emitter attributes should be extracted from (e.g. optical depths ``tau_v``, escape fractions ``fesc``, etc.).

`"stellar"` will cause the model to use a `Stars` object, `"blackhole"` will cause the model to use a `BlackHole/s` object, while `"galaxy"` will cause the model to use a `Galaxy` object. This mechanism is all performed behind the scenes, you should never have to interact directly with any of this beyond setting the emitter string on a model and calling the emitter's ``get_spectra/get_lines`` methods.

As well as the base `EmissionModel` classes there are 3 specialised model classes: `StellarEmissionModel`, `BlackHoleEmissionModel`, and `GalaxyEmissionModel`. These specialised classes are identical to the base `EmissionModel` class but automatically set the ``emitter`` argument to ``"stellar"``, ``"blackhole"``, and ``"galaxy"`` respectively.

We'll cover the specifics of different operations below, but this means the following fake models:

In [ ]:
transmitted = EmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
    fesc=0.3,
    emitter="stellar",
)
nlr = EmissionModel(
    "nlr",
    grid=grid,
    extract="nebular",
    fesc=0.3,
    emitter="blackhole",
)
total = EmissionModel(
    "total",
    combine=(transmitted, nlr),
    emitter="galaxy",
)

are equivalent to:

In [ ]:
from synthesizer.emission_models import (
    BlackHoleEmissionModel,
    GalaxyEmissionModel,
    StellarEmissionModel,
)

transmitted = StellarEmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
    fesc=0.3,
)
nlr = BlackHoleEmissionModel(
    "nlr",
    grid=grid,
    extract="nebular",
    fesc=0.3,
)
total = GalaxyEmissionModel(
    "total",
    combine=(transmitted, nlr),
)


In the rest of this notebook we'll use these specialised classes rather than the base `EmissionModel`.


## Defining an extraction

To define an extraction we simply need to pass a `Grid` to extract from and a key to extract.

In [ ]:
transmitted = StellarEmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
)
print(transmitted)

## Defining a combination

To define a combination we simply pass the models we want combined to the `combine` keyword (along with a label).

In [ ]:
# Define models to combine
linecont = StellarEmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
)
nebular_cont = StellarEmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

# Define the combined model
nebular = StellarEmissionModel("nebular", combine=(linecont, nebular_cont))
print(nebular)

## Defining a transformation

To define a transformation emission model we need a transformer and the model to apply the transformer to (once again along with a label). Here we will create an attenuation transformation where the transformer is an ``AttenuationLaw`` object, see [the attenuation docs](attenuation/attenuation.rst) for more information on these objects.


In [ ]:
attenuated = StellarEmissionModel(
    "attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_to=nebular,
)
print(attenuated)


Some operations will require certain properties are defined on an emitter. These will have some expected name, e.g. `tau_v` or `fesc`, however, you are free to attach any properties you like to an emitter and use them in your models. In this case a dust attenuation transformer requires an optical depth to be defined on the emitter (by default this will look for the ``tau_v`` attribute on the emitter).

To signpost that you want to use a special property you can pass a string as the override on the model. This will cause the spectra generator to look for an attribute on the emitter with this name. Below we will redirect the optical depth from ``tau_v`` to ``ism_tau_v`` as if we had different optical depths certain environments around the stars.

In [ ]:
attenuated = StellarEmissionModel(
    "attenuated",
    dust_curve=PowerLaw(slope=-1),
    apply_to=nebular,
    tau_v="ism_tau_v",
)
print(attenuated)

## Defining a generation

To define a generation model we only need to provide the ``Generator`` class (i.e. a ``Greybody``).

In [ ]:
bb_emission = StellarEmissionModel(
    "blackbody_emission",
    generator=Greybody(30 * kelvin, emissivity=2.0),
)
print(bb_emission)

If we want to scale the generated emission by another emission model we can pass that as a ``scaler`` argument to the ``Generator``.

In [ ]:
scaled_bb_emission = StellarEmissionModel(
    "blackbody_emission",
    generator=Greybody(30 * kelvin, emissivity=2.0, scaler=transmitted),
)
print(scaled_bb_emission)

For the special case of energy balance dust emission you can pass both the intrinsic and attenuated models to scale by the emission reprocessed by dust.

In [ ]:
dust_emission = GalaxyEmissionModel(
    "greybody_emission",
    generator=Greybody(30 * kelvin, emissivity=2.0),
    lum_intrinsic_model=transmitted,
    lum_attenuated_model=attenuated,
)
print(dust_emission)

### Including a mask

A mask can be included in any step by passing `mask_attr`, `mask_thresh`, and `mask_op`.

In [ ]:
masked_transmitted = StellarEmissionModel(
    "masked_transmitted",
    grid=grid,
    extract="transmitted",
    fesc="fesc",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
)

We can also define multiple masks on a single step.
To add more masks, use the ``add_mask`` method, as demonstrated with the [modifying models docs](modify_models.ipynb).
These will be combined with a boolean ``and`` operation at the point of spectra generation.

In [ ]:
masked_transmitted.add_mask("metallicities", "<", 0.01 * dimensionless)
masked_transmitted.add_mask("log10ages", "<", 6 * dimensionless)
print(masked_transmitted)

### Including related models

In many instances you'll have a primary emissions spectra that you wish to calculate, plus some extra emission models that get produced along the way, but do not feature in the main "tree".
To include these models, and ensure their spectra is generated by calls to ``get_spectra`` or ``get_lines`` methods, you can pass them to the ``related_models`` argument at instantiation.

In [ ]:
# Define the combined model
nebular = StellarEmissionModel(
    "nebular", combine=(linecont, nebular_cont), related_models=(transmitted)
)
print(nebular)

### Fixing parameters

When an emission model is used in a ``get_spectra`` method, any parameters the model does not define will be extracted from the emitter.
If you would like to set any of these to fixed values you can pass these to a model at instantiation. They will be stored on the model in a dictionary called ``fixed_parameters``.

For demonstration purposes, below we create an AGN NLR emission model with fixed NLR parameters.
This can also be done with a setter after instantiation (see the model modification [docs](modify_models.ipynb)).

In [ ]:
# Get the NLR and BLR grids
nlr_grid = Grid("test_grid_agn-nlr")

nlr = BlackHoleEmissionModel(
    label="nlr",
    extract="nebular",
    grid=nlr_grid,
    fesc=0.1,  # covering fraction
    ionisation_parameter_nlr=0.01,
    hydrogen_density_nlr=1e4 / cm**3,
    covering_fraction_nlr=0.1,
    velocity_dispersion_nlr=500 * km / s,
)
print(nlr)

### Setting a parameter with a function

In some cases, you may want to compute a parameter dynamically based on other parameters or emitter attributes, rather than fixing it to a constant value. For this purpose, you can use a `ParameterFunction`.

A `ParameterFunction` wraps a callable function that takes one or more parameters as arguments and returns a computed value. When the emission model is used, the function will be called automatically with the required parameters extracted from the emitter, model, or other sources.

To create a `ParameterFunction`, you need to provide:
- A **function** that computes the parameter value
- A **label** for the parameter being set (this is what will be stored in the cache)
- A **list of parameter names** that the function requires as arguments

The function's signature must match the parameter names you provide. Below we demonstrate this by computing the hydrogen density for an NLR based on the black hole mass and a scaling relation.

In [ ]:
from synthesizer.emission_models.utils import ParameterFunction


# Define a function to compute hydrogen density from black hole mass
def compute_hydrogen_density(black_hole_mass):
    """Compute NLR hydrogen density based on black hole mass.

    This is a simplified example using a power-law scaling relation.

    Args:
        black_hole_mass: The black hole mass in solar masses.

    Returns:
        float: The hydrogen density in cm^-3.
    """
    # Simple scaling: n_H = 1e4 * (M_bh / 1e8)^0.5
    return 1e4 * (black_hole_mass / 1e8) ** 0.5


# Create a ParameterFunction that will compute hydrogen_density_nlr
hydrogen_density_func = ParameterFunction(
    func=compute_hydrogen_density,
    sets="hydrogen_density_nlr",  # The parameter this function sets
    func_args=["black_hole_mass"],  # Arguments the function needs
)

# Now use it in an emission model
nlr_dynamic = BlackHoleEmissionModel(
    label="nlr_dynamic",
    extract="nebular",
    grid=nlr_grid,
    fesc=0.1,
    ionisation_parameter_nlr=0.01,
    hydrogen_density_nlr=hydrogen_density_func,  # Use the function!
    covering_fraction_nlr=0.1,
    velocity_dispersion_nlr=500 * km / s,
)

print(nlr_dynamic)
print("\nFixed parameters:")
print(nlr_dynamic.fixed_parameters)

When `get_spectra` or `get_lines` is called, the `ParameterFunction` will:
1. Extract the required parameters (in this case `black_hole_mass`) from the emitter or model's fixed parameters
2. Call the function with these parameters
3. Cache the computed value in the `model_param_cache` dictionary on the emitter (under the model's label)
4. Return the computed value to be used in the grid extraction

The computed parameter is stored in `emitter.model_param_cache[model.label][parameter_name]`, allowing you to inspect the values that were used for each model after spectra generation.

This approach is particularly useful for:
- Implementing scaling relations between physical properties
- Computing derived quantities from multiple parameters
- Creating models where parameters depend on emitter properties in complex ways

The function can depend on any parameter that would normally be extracted from the emitter, including other fixed parameters or attributes on the emitter object.

## Using post processing functions

You may want to apply some post processing to your resulting spectra that isn't offered by synthesizer.
In these cases you can pass a list of "post processing functions" to the ``EmissionModel``.
These functions will be called in the order they are passed, after all spectra have been generated. 

These post processing functions must take the following arguments:

- A dictionary of spectra / lines, where the dictionary will be filled by key-value pairs of emission model labels and the spectra / lines they produce (``Sed``/``LineCollection``) ``{<model.label>: <spectra / lines>}``. 
- A dictionary of galaxy / component objects, where the dictionary will be ``{"stellar": Stars, "blackhole": BlackHoles, "galaxy": Galaxy}``.
- An ``EmissionModel``. 

Each post processing function should also return the dictionary of spectra / lines with the modified values.

When ``get_spectra`` is called, and all spectra have been generated, any post processing functions will be called with these arguments (in the order listed above).

Below we define a **fake** function to perform some arbitrary mass scaling of the spectra. 

## Saving or not saving the emission

By default all emissions produced by a model will be attached to an emitter after generation. If you would like to instead discard a spectra (but still have it generated for making other spectra) you can pass `save=False` to the emission model at instantiation.

In [ ]:
throw_away = StellarEmissionModel(
    "throw_away",
    combine=(linecont, nebular_cont),
    save=False,
)

## Putting it all together

In the example below we demonstrate how to construct a complex model piece by piece. 
This will construct the `TotalEmission` model with the [Charlot&Fall+2000](https://ui.adsabs.harvard.edu/abs/2000ApJ...539..718C/abstract) like attenuation operation explicitly.

Here we'll use the base `EmissionModel` class to explictly show the setting of the `emitter` argument. Notice how we associate the dust emission and total emission on the galaxy rather than the stellar component.

In [ ]:
# Define the extractions
transmitted = EmissionModel(
    "transmitted",
    grid=grid,
    extract="transmitted",
    fesc=0.3,
    emitter="stellar",
)
escaped = EmissionModel(
    "escaped", grid=grid, extract="transmitted", fesc=0.7, emitter="stellar"
)
linecont = EmissionModel(
    "linecont",
    grid=grid,
    extract="linecont",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    fesc=0.7,
    emitter="stellar",
    save=False,
)
nebular_cont = EmissionModel(
    "nebular_continuum",
    grid=grid,
    extract="nebular_continuum",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    emitter="stellar",
    save=False,
)

# Combine the extractions
nebular = EmissionModel(
    "nebular", combine=(linecont, nebular_cont), emitter="stellar"
)
reprocessed = EmissionModel(
    "reprocessed", combine=(nebular, transmitted), emitter="stellar"
)

# Apply the young and old dust attenuation
young_attenuated = EmissionModel(
    "young_attenuated",
    dust_curve=PowerLaw,
    apply_to=reprocessed,
    tau_v=("tau_v", 0.67),
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op="<",
    emitter="stellar",
    save=False,
)
old_attenuated = EmissionModel(
    "old_attenuated",
    dust_curve=PowerLaw,
    apply_to=reprocessed,
    tau_v="tau_v",
    mask_attr="log10ages",
    mask_thresh=7 * dimensionless,
    mask_op=">=",
    emitter="stellar",
    save=False,
)

# And combine them into a single attenuated spectra
attenuated = EmissionModel(
    "attenuated", combine=(young_attenuated, old_attenuated), emitter="stellar"
)
emergent = EmissionModel(
    "emergent", combine=(attenuated, escaped), emitter="stellar"
)

# Create a dust emission model
dust_emission = EmissionModel(
    "dust_emission",
    generator=Greybody(temperature=30 * kelvin, emissivity=2.0),
    lum_intrinsic_model=reprocessed,
    lum_attenuated_model=emergent["attenuated"],
    emitter="galaxy",
)

# And bring everything together into the total emission
total = EmissionModel(
    "total", combine=(emergent, dust_emission), emitter="galaxy"
)

total.plot_emission_tree()
print(total)